## Model Deployment

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import tensorflow as tf
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.layers import Conv2D, MaxPool2D, Dense, Flatten, InputLayer, BatchNormalization, Input, Dropout, RandomFlip, Resizing, Rescaling
from tensorflow.keras.metrics import BinaryAccuracy, FalsePositives, FalseNegatives, TruePositives, TrueNegatives, Precision, Recall, AUC
from tensorflow.keras.losses import BinaryCrossentropy
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import CSVLogger, EarlyStopping, LearningRateScheduler, ReduceLROnPlateau
from tensorflow.keras.regularizers  import L2, L1

from sklearn.metrics import confusion_matrix, roc_curve

import tensorflow_datasets as tfds


from collections import defaultdict
from PIL import Image

import os
import io
import random

In [2]:
dataset =  tf.data.Dataset.load(r"C:\Users\jorge\DataSets\MalariaDS")

### Parameters definition

In [3]:
CLASS_NAMES = ["infected", "uninfected"]

CONFIGURATION = {
    "BATCH_SIZE" : 32,
    "IM_HEIGHT" : 256,
    "IM_WIDTH" : 256,
    "NORMALIZATION" : 255,
    "LEARNING_RATE": 0.001,
    "N_EPOCHS": 2,
    "DROPOUT_RATE": 0.0,
    "REGULARIZATION_RATE": 0.0,
    "N_FILTERS": 6,
    "KERNEL_SIZE": 3,
    "N_STRIDES": 1,
    "POOL_SIZE": 2,
    "N_DENSE_1": 100,
    "N_DENSE_2": 10,
    "NUM_CLASSES":2
}

### Rescaling and Normalizing

In [4]:
resize_rescale_layers = Sequential([
    Resizing(CONFIGURATION["IM_HEIGHT"], CONFIGURATION["IM_WIDTH"], interpolation="bilinear"),           # Interpolation method used: Bilinear
    Rescaling(1./CONFIGURATION["NORMALIZATION"])])                                                       # Normalization factor            

@tf.function
def map_fn(images, labels):
    processed_images = resize_rescale_layers(images)
    return processed_images, labels

In [5]:
dataset = dataset.map(map_fn, num_parallel_calls = tf.data.AUTOTUNE).batch(CONFIGURATION["BATCH_SIZE"])

### Data splitting

In [6]:
def splits(dataset, TRAIN_RATIO, VAL_RATIO, TEST_RATIO, seed = None):
  DATASET_SIZE = len(dataset)

  if seed is not None:
    dataset = dataset.shuffle(DATASET_SIZE, seed=seed)
  else:
    dataset = dataset.shuffle(DATASET_SIZE)

  train_dataset = dataset.take(int(TRAIN_RATIO*DATASET_SIZE))

  val_test_dataset = dataset.skip(int(TRAIN_RATIO*DATASET_SIZE))
  val_dataset = val_test_dataset.take(int(VAL_RATIO*DATASET_SIZE))

  test_dataset = val_test_dataset.skip(int(VAL_RATIO*DATASET_SIZE))
  return train_dataset, val_dataset, test_dataset

TRAIN_RATIO = 0.8
VAL_RATIO = 0.1
TEST_RATIO = 0.1

In [7]:
train_dataset, val_dataset, test_dataset = splits(dataset, TRAIN_RATIO, VAL_RATIO, TEST_RATIO)

### Data optimization

In [8]:
train_dataset = (
    train_dataset
    .shuffle(buffer_size = 1024, reshuffle_each_iteration = True)
    .prefetch(tf.data.AUTOTUNE)
)

val_dataset = (
    val_dataset
    .shuffle(buffer_size = 32)
)

### Callbacks

#### CSV logger

In [9]:
csv_callback = CSVLogger(r"C:\Users\jorge\OneDrive\Imágenes\Documentos\Malaria Detection\Model logs\logs.csv", 
                        separator = ",",
                        append = True)

#### EarlyStopping

In [10]:
es_callback = EarlyStopping(
    monitor = "val_loss",     
    min_delta = 0,                        #An absolute change of less than min_delta, will count as no improvement.
    patience = 5,                         #Number of epochs with no improvement after which training will be stopped.
    verbose = 0,          
    mode = "auto",                        #In min mode, training will stop when the quantity monitored has stopped decreasing; in max mode it will stop when the quantity monitored has stopped increasing.
    baseline = None,                      #Baseline value for the monitored quantity. Training will stop if the model doesn't show improvement over the baseline.
    restore_best_weights = False          #Whether to restore model weights from the epoch with the best value of the monitored quantity. If False, the model weights obtained at the last step of training are used.
)

#### Learning Rate Scheduler

In [26]:
def scheduler(epoch, lr):
    if epoch <= 1:
        learning_rate = lr
    else:
        learning_rate = lr * tf.math.exp(-0.1)
    return learning_rate
  
scheduler_callback = LearningRateScheduler(scheduler, verbose = 1)

#### ReduceLROnPlateau

In [12]:
plateau_callback = ReduceLROnPlateau(monitor = "val_accuracy", 
                                     factor = 0.3, 
                                     patience = 5, 
                                     verbose = 1)

### Model definition

#### Custom loss class

This is done in case the custom loss functions needs to be modified. For now it will remain the same, but is a good practice to define this function in case is necessary to change it in future phases of the analysis. 

In [13]:
class CustomBCE(tf.keras.losses.Loss):
  def __init__(self, FACTOR):
    super(CustomBCE, self).__init__()
    self.FACTOR = FACTOR

  def call(self, y_true, y_pred):
    bce = BinaryCrossentropy()
    return bce(y_true, y_pred)* self.FACTOR

#### Custom metrics class

In [21]:
class CustomAccuracy(tf.keras.metrics.Metric):
  def __init__(self, name = 'Custom_Accuracy', FACTOR = 1):
    super(CustomAccuracy, self).__init__()
    self.FACTOR = FACTOR
    self.accuracy = self.add_weight(name = name, initializer = 'zeros')


  def update_state(self, y_true, y_pred, sample_weight = None):
    output = binary_accuracy(tf.cast(y_true, dtype = tf.float32), y_pred)*self.FACTOR
    self.accuracy.assign(tf.math.count_nonzero(output, dtype = tf.float32)/tf.cast(len(output), dtype = tf.float32))

  def result(self):
    return self.accuracy

  def reset_states(self):
    self.accuracy.assign(0.)

FACTOR = 1

#### Model metrics definition

In [15]:
metrics = [TruePositives(name='tp'),FalsePositives(name='fp'), TrueNegatives(name='tn'), FalseNegatives(name='fn'), 
            BinaryAccuracy(name='accuracy'), Precision(name='precision'), Recall(name='recall'), AUC(name='auc')]

### Model Architecture

#### Feature extractor (sequential API)

In [16]:
feature_extractor_seq_model = tf.keras.Sequential([
    InputLayer(input_shape = (CONFIGURATION["IM_HEIGHT"], CONFIGURATION["IM_WIDTH"], 3)),

    Conv2D(filters = 6, kernel_size = 3, strides=1, padding='valid', activation = 'relu'),
    BatchNormalization(),
    MaxPool2D (pool_size = 2, strides= 2),

    Conv2D(filters = 16, kernel_size = 3, strides=1, padding='valid', activation = 'relu'),
    BatchNormalization(),
    MaxPool2D (pool_size = 2, strides= 2),
])

In [17]:
feature_extractor_seq_model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 254, 254, 6)       168       
                                                                 
 batch_normalization (Batch  (None, 254, 254, 6)       24        
 Normalization)                                                  
                                                                 
 max_pooling2d (MaxPooling2  (None, 127, 127, 6)       0         
 D)                                                              
                                                                 
 conv2d_1 (Conv2D)           (None, 125, 125, 16)      880       
                                                                 
 batch_normalization_1 (Bat  (None, 125, 125, 16)      64        
 chNormalization)                                                
                                                      

#### Callable Model (functional API)

In [18]:
func_input = Input(shape = (CONFIGURATION["IM_HEIGHT"], CONFIGURATION["IM_WIDTH"], 3), name = "Input Image")

x = feature_extractor_seq_model(func_input)

x = Flatten()(x)

x = Dense(100, activation = "relu")(x)
x = BatchNormalization()(x)

x = Dense(10, activation = "relu")(x)
x = BatchNormalization()(x)

func_output = Dense(1, activation = "sigmoid")(x)

lenet_model = Model(func_input, func_output, name = "Lenet_Model")

In [19]:
lenet_model.summary()

Model: "Lenet_Model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 Input Image (InputLayer)    [(None, 256, 256, 3)]     0         
                                                                 
 sequential_1 (Sequential)   (None, 62, 62, 16)        1136      
                                                                 
 flatten (Flatten)           (None, 61504)             0         
                                                                 
 dense (Dense)               (None, 100)               6150500   
                                                                 
 batch_normalization_2 (Bat  (None, 100)               400       
 chNormalization)                                                
                                                                 
 dense_1 (Dense)             (None, 10)                1010      
                                                       

### Model training

In [22]:
lenet_model.compile(optimizer = Adam(learning_rate = CONFIGURATION['LEARNING_RATE']),
      loss = BinaryCrossentropy(CustomBCE(FACTOR)),        # Custom loss class defined above
      metrics = metrics)                             # Metriics list defined above. For now the class that was defined for the accuracy won´t be used. 

In [27]:
history = lenet_model.fit(
    train_dataset,
    validation_data = val_dataset,
    epochs = CONFIGURATION['N_EPOCHS'],
    verbose = 1,
    callbacks = [csv_callback, es_callback, scheduler_callback, plateau_callback]
    )


Epoch 1: LearningRateScheduler setting learning rate to 0.0010000000474974513.
Epoch 1/2


c:\Users\jorge\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\backend.py:5805: UserWarning: "`binary_crossentropy` received `from_logits=True`, but the `output` argument was produced by a Sigmoid activation and thus does not represent logits. Was this intended?
  output, from_logits = _get_logits(


109/689 [===>..........................] - ETA: 8:26 - loss: 0.6345 - tp: 1202.0000 - fp: 613.0000 - tn: 1086.0000 - fn: 587.0000 - accuracy: 0.6560 - precision: 0.6623 - recall: 0.6719 - auc: 0.7063

KeyboardInterrupt: 

### Biblbiography

Learning rate scheduler      -       https://mxnet.apache.org/versions/1.9.1/api/python/docs/tutorials/packages/gluon/training/learning_rates/learning_rate_schedules_advanced.html    
BinaryCrossEntropy           -       https://towardsdatascience.com/understanding-binary-cross-entropy-log-loss-a-visual-explanation-a3ac6025181a                     